test code to see xrv lib

In [96]:
img = skimage.io.imread(r"C:\Users\Ayushman\OneDrive - Nanyang Technological University\FYP\NIH_CXR_Dataset\CXR_ALL_FLAT\00000001_000.png")
xrv.datasets.normalize(img, 255)
img

array([[202, 199, 195, ...,   5,   2,   0],
       [199, 196, 195, ...,   5,   2,   0],
       [196, 194, 193, ...,   5,   2,   0],
       ...,
       [255, 255, 255, ...,   0,   0,   0],
       [255, 255, 254, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0]],
      shape=(1024, 1024), dtype=uint8)

In [2]:
import torchxrayvision as xrv
import skimage, torch, torchvision

# Prepare the image:
img = skimage.io.imread(r"C:\Users\Ayushman\OneDrive - Nanyang Technological University\FYP\NIH_CXR_Dataset\CXR_ALL_FLAT\00021091_002.png")
img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
# img = img.mean(2)[None, ...] # Make single color channel

transform = torchvision.transforms.Compose([
    xrv.datasets.XRayCenterCrop(),
    xrv.datasets.XRayResizer(224),
])


img = img[None, :, :]
img = transform(img)
img = torch.from_numpy(img)

# Load model and process image
model = xrv.models.DenseNet(weights="densenet121-res224-all")
outputs = model(img[None,...]) # or model.features(img[None,...])

# Print results
dict(zip(model.pathologies, outputs[0].detach().numpy()))


{'Atelectasis': np.float32(0.57338643),
 'Consolidation': np.float32(0.5580266),
 'Infiltration': np.float32(0.5303836),
 'Pneumothorax': np.float32(0.5043328),
 'Edema': np.float32(0.10917771),
 'Emphysema': np.float32(0.5009655),
 'Fibrosis': np.float32(0.54985535),
 'Effusion': np.float32(0.6399379),
 'Pneumonia': np.float32(0.09603219),
 'Pleural_Thickening': np.float32(0.534311),
 'Cardiomegaly': np.float32(0.3065372),
 'Nodule': np.float32(0.5630389),
 'Mass': np.float32(0.5961195),
 'Hernia': np.float32(0.0095163295),
 'Lung Lesion': np.float32(0.6115719),
 'Fracture': np.float32(0.5213179),
 'Lung Opacity': np.float32(0.7580718),
 'Enlarged Cardiomediastinum': np.float32(0.56434613)}

tensor([[0.5845, 0.5440, 0.5108, 0.5221, 0.5296, 0.5219, 0.5100, 0.5407, 0.6170,
         0.5343, 0.5324, 0.5211, 0.5590, 0.2375, 0.2073, 0.5179, 0.6812, 0.5643]],
       grad_fn=<IndexPutBackward0>)

Load NIH Dataset - only PA view

In [46]:
nih_dataset = xrv.datasets.NIH_Dataset(imgpath=r"C:\Users\Ayushman\OneDrive - Nanyang Technological University\FYP\NIH_CXR_Dataset\CXR_ALL_FLAT", 
                         csvpath=r"C:\Users\Ayushman\OneDrive - Nanyang Technological University\FYP\NIH_CXR_Dataset\Data_Entry_2017.csv",
                         bbox_list_path=r"C:\Users\Ayushman\OneDrive - Nanyang Technological University\FYP\NIH_CXR_Dataset\BBox_List_2017.csv", 
                         views=["PA"],
                         unique_patients=False)
nih_dataset

{'Atelectasis': {np.float32(0.0): 61582, np.float32(1.0): 5728},
 'Cardiomegaly': {np.float32(0.0): 65747, np.float32(1.0): 1563},
 'Consolidation': {np.float32(0.0): 65789, np.float32(1.0): 1521},
 'Edema': {np.float32(0.0): 67034, np.float32(1.0): 276},
 'Effusion': {np.float32(0.0): 60721, np.float32(1.0): 6589},
 'Emphysema': {np.float32(0.0): 65811, np.float32(1.0): 1499},
 'Fibrosis': {np.float32(0.0): 65902, np.float32(1.0): 1408},
 'Hernia': {np.float32(0.0): 67118, np.float32(1.0): 192},
 'Infiltration': {np.float32(0.0): 57957, np.float32(1.0): 9353},
 'Mass': {np.float32(0.0): 63743, np.float32(1.0): 3567},
 'Nodule': {np.float32(0.0): 63133, np.float32(1.0): 4177},
 'Pleural_Thickening': {np.float32(0.0): 64892, np.float32(1.0): 2418},
 'Pneumonia': {np.float32(0.0): 66680, np.float32(1.0): 630},
 'Pneumothorax': {np.float32(0.0): 63903, np.float32(1.0): 3407}}


NIH_Dataset num_samples=67310 views=['PA'] data_aug=None

In [59]:
import torchxrayvision as xrv
import pandas as pd
import numpy as np
import os
import random
import shutil

# Optional: Merge all NIH image subfolders into one flat directory
src_base = "C:/Users/Ayushman/OneDrive - Nanyang Technological University/FYP/NIH_CXR_Dataset"

# Load the NIH metadata using pandas
csv_path = os.path.join(src_base, "Data_Entry_2017.csv")
df = pd.read_csv(csv_path)

# Filter for PA view only
df = df[df["View Position"] == "PA"]

# Define target conditions
target_conditions = ["Pneumonia", "Effusion", "Cardiomegaly", "Infiltration", "Atelectasis"]

# Create binary columns for each target condition
def label_contains(label_str, condition):
    return int(condition in str(label_str).split("|"))

for condition in target_conditions:
    df[condition] = df["Finding Labels"].apply(lambda x: label_contains(x, condition))

# Select up to N samples per condition
samples_per_class = 200
selected_dfs = []
class_distribution = {}

for condition in target_conditions:
    pos_samples = df[df[condition] == 1]
    selected = pos_samples.sample(n=min(samples_per_class, len(pos_samples)), random_state=42)
    selected_dfs.append(selected)
    class_distribution[condition] = len(selected)

# Combine and deduplicate
selected_df = pd.concat(selected_dfs).drop_duplicates(subset="Image Index")
selected_df = selected_df.reset_index(drop=True)

print(f"Balanced subset created with {len(selected_df)} unique images across {len(target_conditions)} classes.")
print("\nClass distribution in sampled subset:")
for condition, count in class_distribution.items():
    print(f"{condition}: {count} samples")

# Save selected metadata to CSV
subset_csv_path = os.path.join(src_base, "sampled_subset_metadata.csv")
selected_df.to_csv(subset_csv_path, index=False)
print(f"\nMetadata CSV saved to: {subset_csv_path}")


Balanced subset created with 988 unique images across 5 classes.

Class distribution in sampled subset:
Pneumonia: 200 samples
Effusion: 200 samples
Cardiomegaly: 200 samples
Infiltration: 200 samples
Atelectasis: 200 samples

Metadata CSV saved to: C:/Users/Ayushman/OneDrive - Nanyang Technological University/FYP/NIH_CXR_Dataset\sampled_subset_metadata.csv


In [115]:
import os
import pandas as pd
import torch
import torchxrayvision as xrv
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm

# Config
CSV_PATH = "C:/Users/Ayushman/OneDrive - Nanyang Technological University/FYP/NIH_CXR_Dataset/sampled_subset_metadata.csv"
IMAGE_DIR = r"C:\Users\Ayushman\OneDrive - Nanyang Technological University\FYP\NIH_CXR_Dataset\CXR_ALL_FLAT"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load CSV
df = pd.read_csv(CSV_PATH)

# Load pretrained model
model = xrv.models.DenseNet(weights="densenet121-res224-all")
# model = model.to(DEVICE)
# model.eval()

# Define image preprocessing
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor()  # for grayscale images
# ])

transform = torchvision.transforms.Compose([
    xrv.datasets.XRayCenterCrop(),
    xrv.datasets.XRayResizer(224),
])


# Map torchxrayvision class indices to disease names
disease_labels = model.pathologies

# Prepare results
results = []

# Inference
with torch.no_grad():
    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            img_path = os.path.join(IMAGE_DIR, row["Image Index"])  # Adjust column name if needed
            img = skimage.io.imread(img_path)
            img = img[None, :, :]
            img = transform(img)
            img = torch.from_numpy(img)
            # img_tensor = transform(img).unsqueeze(0).to(DEVICE)
            preds = model(img[None, ...])
            preds_np = preds.cpu().numpy().flatten()

            # Store results
            pred_dict = {"image_filename": row["Image Index"]}
            pred_dict.update({disease_labels[i]: preds_np[i] for i in range(len(disease_labels))})
            results.append(pred_dict)
        except:
            print("Could not run inference for: ", row["Image Index"])

# Save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("inference_results.csv", index=False)
print("Inference complete. Results saved to inference_results.csv")


  3%|▎         | 25/988 [00:29<17:49,  1.11s/it]

Could not run inference for:  00004808_003.png


 18%|█▊        | 175/988 [03:31<15:28,  1.14s/it]

Could not run inference for:  00014211_003.png


 33%|███▎      | 328/988 [06:32<12:49,  1.17s/it]

...

Could not run inference for:  00000955_002.png


 93%|█████████▎| 919/988 [18:01<01:21,  1.19s/it]

Could not run inference for:  00007156_000.png


100%|██████████| 988/988 [19:24<00:00,  1.18s/it]

Inference complete. Results saved to inference_results.csv


In [3]:
import os
import pandas as pd
import torch
import torchxrayvision as xrv
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm

# Config
CSV_PATH = "C:/Users/Ayushman/OneDrive - Nanyang Technological University/FYP/NIH_CXR_Dataset/sampled_subset_metadata.csv"
IMAGE_DIR = r"C:\Users\Ayushman\OneDrive - Nanyang Technological University\FYP\NIH_CXR_Dataset\CXR_ALL_FLAT"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load CSV
df = pd.read_csv(CSV_PATH)

# Load pretrained model
model = xrv.models.DenseNet(weights="densenet121-res224-chex")
# model = model.to(DEVICE)
# model.eval()

# Define image preprocessing
transform = torchvision.transforms.Compose([
    xrv.datasets.XRayCenterCrop(),
    xrv.datasets.XRayResizer(224),
])


# Map torchxrayvision class indices to disease names
disease_labels = model.pathologies

# Prepare results
results = []

# Inference
with torch.no_grad():
    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            img_path = os.path.join(IMAGE_DIR, row["Image Index"])  # Adjust column name if needed
            img = skimage.io.imread(img_path)
            img = img[None, :, :]
            img = transform(img)
            img = torch.from_numpy(img)
            # img_tensor = transform(img).unsqueeze(0).to(DEVICE)
            preds = model(img[None, ...])
            preds_np = preds.cpu().numpy().flatten()

            # Store results
            pred_dict = {"image_filename": row["Image Index"]}
            pred_dict.update({disease_labels[i]: preds_np[i] for i in range(len(disease_labels))})
            results.append(pred_dict)
        except:
            print("Could not run inference for: ", row["Image Index"])

# Save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("inference_results_chex.csv", index=False)
print("Inference complete. Results saved to inference_results.csv")


  3%|▎         | 25/988 [00:15<10:28,  1.53it/s]

Could not run inference for:  00004808_003.png


 18%|█▊        | 175/988 [02:07<09:06,  1.49it/s]

Could not run inference for:  00014211_003.png


 33%|███▎      | 328/988 [04:02<06:30,  1.69it/s]

...

Could not run inference for:  00000955_002.png


 93%|█████████▎| 919/988 [10:25<00:47,  1.46it/s]

Could not run inference for:  00007156_000.png


100%|██████████| 988/988 [11:04<00:00,  1.49it/s]

Inference complete. Results saved to inference_results.csv


In [5]:
import os
import pandas as pd
import torch
import torchxrayvision as xrv
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm

# Config
CSV_PATH = "C:/Users/Ayushman/OneDrive - Nanyang Technological University/FYP/NIH_CXR_Dataset/sampled_subset_metadata.csv"
IMAGE_DIR = r"C:\Users\Ayushman\OneDrive - Nanyang Technological University\FYP\NIH_CXR_Dataset\CXR_ALL_FLAT"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load CSV
df = pd.read_csv(CSV_PATH)

# Load pretrained model
model = xrv.models.DenseNet(weights="densenet121-res224-mimic_nb")
# model = model.to(DEVICE)
# model.eval()

# Define image preprocessing
transform = torchvision.transforms.Compose([
    xrv.datasets.XRayCenterCrop(),
    xrv.datasets.XRayResizer(224),
])


# Map torchxrayvision class indices to disease names
disease_labels = model.pathologies

# Prepare results
results = []

# Inference
with torch.no_grad():
    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            img_path = os.path.join(IMAGE_DIR, row["Image Index"])  # Adjust column name if needed
            img = skimage.io.imread(img_path)
            img = img[None, :, :]
            img = transform(img)
            img = torch.from_numpy(img)
            # img_tensor = transform(img).unsqueeze(0).to(DEVICE)
            preds = model(img[None, ...])
            preds_np = preds.cpu().numpy().flatten()

            # Store results
            pred_dict = {"image_filename": row["Image Index"]}
            pred_dict.update({disease_labels[i]: preds_np[i] for i in range(len(disease_labels))})
            results.append(pred_dict)
        except:
            print("Could not run inference for: ", row["Image Index"])


# Save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("inference_results_mimic_nb.csv", index=False)
print("Inference complete. Results saved to inference_results.csv")


  3%|▎         | 25/988 [00:06<03:59,  4.03it/s]

Could not run inference for:  00004808_003.png


 18%|█▊        | 175/988 [03:19<24:11,  1.79s/it]

Could not run inference for:  00014211_003.png


 33%|███▎      | 328/988 [06:56<14:17,  1.30s/it]

...

Could not run inference for:  00000955_002.png


 93%|█████████▎| 919/988 [26:55<02:28,  2.16s/it]

Could not run inference for:  00007156_000.png


100%|██████████| 988/988 [29:22<00:00,  1.78s/it]

Inference complete. Results saved to inference_results.csv


In [4]:
import os
import pandas as pd
import torch
import torchxrayvision as xrv
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm

# Config
CSV_PATH = "C:/Users/Ayushman/OneDrive - Nanyang Technological University/FYP/NIH_CXR_Dataset/sampled_subset_metadata.csv"
IMAGE_DIR = r"C:\Users\Ayushman\OneDrive - Nanyang Technological University\FYP\NIH_CXR_Dataset\CXR_ALL_FLAT"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load CSV
df = pd.read_csv(CSV_PATH)

# Load pretrained model
model = xrv.models.DenseNet(weights="densenet121-res224-nih")
# model = model.to(DEVICE)
# model.eval()

# Define image preprocessing
transform = torchvision.transforms.Compose([
    xrv.datasets.XRayCenterCrop(),
    xrv.datasets.XRayResizer(224),
])


# Map torchxrayvision class indices to disease names
disease_labels = model.pathologies

# Prepare results
results = []

# Inference
with torch.no_grad():
    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            img_path = os.path.join(IMAGE_DIR, row["Image Index"])  # Adjust column name if needed
            img = skimage.io.imread(img_path)
            img = img[None, :, :]
            img = transform(img)
            img = torch.from_numpy(img)
            # img_tensor = transform(img).unsqueeze(0).to(DEVICE)
            preds = model(img[None, ...])
            preds_np = preds.cpu().numpy().flatten()

            # Store results
            pred_dict = {"image_filename": row["Image Index"]}
            pred_dict.update({disease_labels[i]: preds_np[i] for i in range(len(disease_labels))})
            results.append(pred_dict)
        except:
            print("Could not run inference for: ", row["Image Index"])
# Save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("inference_results_nih.csv", index=False)
print("Inference complete. Results saved to inference_results.csv")


  3%|▎         | 25/988 [00:12<08:27,  1.90it/s]

Could not run inference for:  00004808_003.png


 18%|█▊        | 175/988 [01:32<07:32,  1.80it/s]

Could not run inference for:  00014211_003.png


 33%|███▎      | 328/988 [02:57<07:21,  1.49it/s]

...

Could not run inference for:  00000955_002.png


 93%|█████████▎| 919/988 [08:34<00:40,  1.68it/s]

Could not run inference for:  00007156_000.png


100%|██████████| 988/988 [09:13<00:00,  1.78it/s]

Inference complete. Results saved to inference_results.csv


In [103]:
model = xrv.models.DenseNet(weights="densenet121-res224-mimic_nb") #Cchex , mimic_nb, resnet50-res512-chex, resnet50-res512-mimic_ch
model.pathologies


['Atelectasis',
 'Consolidation',
 '',
 'Pneumothorax',
 'Edema',
 '',
 '',
 'Effusion',
 'Pneumonia',
 '',
 'Cardiomegaly',
 '',
 '',
 '',
 'Lung Lesion',
 'Fracture',
 'Lung Opacity',
 'Enlarged Cardiomediastinum']

In [ ]:
import pandas as pd

targets = ["Pneumonia", "Effusion", "Cardiomegaly", "Infiltration", "Atelectasis"]

model_csvs = {
    "NIH": "inference_results_nih.csv",
    "CheXpert": "inference_results_chex.csv",
    "MIMIC": "inference_results_mimic_nb.csv",
    "All": "inference_results.csv"
}

summary = {}

for model_name, path in model_csvs.items():
    df = pd.read_csv(path)
    summary[model_name] = [df[label].mean() for label in targets]

summary_df = pd.DataFrame(summary, index=targets)
print(summary_df)
summary_df.to_csv("model_mean_predictions.csv")
